In [5]:
import csv
from socket import MSG_MCAST
from urllib.parse import quote, quote_plus
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 민성 20년 08월, 09월, 10월, 11월
# 하영 20년 12월, 21년 01월, 02월
# 창석 21년 03월, 04월, 05월
# 승주 21년 06월, 07월
# 수원 21년 08월

In [8]:
strDate = '20201101'
endDate = '20201130'

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}


filename = f"daum_news_{strDate}-{endDate}.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

dt_index = pd.date_range(start=strDate, end=endDate)
dt_list = dt_index.strftime("%Y%m%d").tolist()
max_page = 10000 # 뉴스 페이지 탭 수 지정


for i in dt_list:
    print('날짜',i)
    for j in range(1, max_page):
        main_url = f"https://news.daum.net/breakingnews/?page={j}&regDate={i}" # url 입력
        res = requests.get(main_url, headers=headers)
 
        val_url = f"https://news.daum.net/breakingnews/?page={j+1}&regDate={i}"
        val = requests.get(val_url, headers=headers)


        if (res.status_code == 200) & (val.status_code == 200):
            print('page : ',j, main_url, 'status:',res.status_code)
            soup = BeautifulSoup(res.text, "lxml") # soup으로 저장
            val_soup = BeautifulSoup(val.text, "lxml")

            main = soup.find("ul", attrs={"class":"list_news2 list_allnews"})
            val_main = val_soup.find("ul", attrs={"class":"list_news2 list_allnews"})

            if main != val_main:
                news = main.find_all("strong", attrs={"class":"tit_thumb"})
                cnt = 0
                for new in news:
                    urls = new.select_one("a")["href"]# 페이지에 나와있는 뉴스 URL 변수 입력
                    # print(urls)
                    result = requests.get(urls, headers=headers)         # request 로 다시 개별 뉴스 접속
                    if result.status_code == 200:
                        news_soup = BeautifulSoup(result.text, "lxml")
                        # 뉴스 제목, 발행시간, 기사본문 저장
                        title = news_soup.find("h3", attrs={"tit_view"}).get_text().strip()
                        pubdate = news_soup.find("span", attrs={"num_date"}).get_text().strip()
                        text = news_soup.find("div", attrs={"news_view"}).get_text().strip()
                        cnt += 1
                        print(j,'of',cnt,'번째 기사')
                        # print(i,j,'of',cnt,'번째 기사', urls,'status:', result.status_code)
                        writer.writerow([cnt, title, pubdate, urls, text])
                    else:
                        print(i,j,'of',cnt,'번째 기사','error_code :',result.status_code, urls)
                        pass
            else:
                news = main.find_all("strong", attrs={"class":"tit_thumb"})
                cnt = 0
                for new in news:
                    urls = new.select_one("a")["href"]# 페이지에 나와있는 뉴스 URL 변수 입력
                    # print(urls)
                    result = requests.get(urls, headers=headers)         # request 로 다시 개별 뉴스 접속
                    if result.status_code == 200:
                        news_soup = BeautifulSoup(result.text, "lxml")
                        # 뉴스 제목, 발행시간, 기사본문 저장
                        title = news_soup.find("h3", attrs={"tit_view"}).get_text().strip()
                        pubdate = news_soup.find("span", attrs={"num_date"}).get_text().strip()
                        text = news_soup.find("div", attrs={"news_view"}).get_text().strip()
                        cnt += 1
                        print(j,'of',cnt,'번째 기사')
                        # print(i,j,'of',cnt,'번째 기사', urls,'status:', result.status_code)
                        writer.writerow([cnt, title, pubdate, urls, text])
                    else:
                        print(i,j,'of',cnt,'번째 기사','error_code :',result.status_code, urls)
                        pass
               
                break
                    
        else:
            print(i,'page : ',j,'error_code :',res.status_code, main_url)
            pass

날짜 20201101
page :  1 https://news.daum.net/breakingnews/?page=1&regDate=20201101 status: 200
1 of 1 번째 기사
1 of 2 번째 기사
1 of 3 번째 기사
1 of 4 번째 기사
1 of 5 번째 기사
1 of 6 번째 기사
1 of 7 번째 기사
1 of 8 번째 기사
1 of 9 번째 기사
1 of 10 번째 기사
1 of 11 번째 기사
1 of 12 번째 기사
1 of 13 번째 기사
1 of 14 번째 기사
1 of 15 번째 기사
page :  2 https://news.daum.net/breakingnews/?page=2&regDate=20201101 status: 200
2 of 1 번째 기사
2 of 2 번째 기사
2 of 3 번째 기사
2 of 4 번째 기사
2 of 5 번째 기사
2 of 6 번째 기사
2 of 7 번째 기사
2 of 8 번째 기사
2 of 9 번째 기사
2 of 10 번째 기사
2 of 11 번째 기사
2 of 12 번째 기사
2 of 13 번째 기사
2 of 14 번째 기사
2 of 15 번째 기사
page :  3 https://news.daum.net/breakingnews/?page=3&regDate=20201101 status: 200
3 of 1 번째 기사
3 of 2 번째 기사
3 of 3 번째 기사
3 of 4 번째 기사
3 of 5 번째 기사
3 of 6 번째 기사
3 of 7 번째 기사
3 of 8 번째 기사
3 of 9 번째 기사
3 of 10 번째 기사
3 of 11 번째 기사
3 of 12 번째 기사
3 of 13 번째 기사
3 of 14 번째 기사
3 of 15 번째 기사
page :  4 https://news.daum.net/breakingnews/?page=4&regDate=20201101 status: 200
4 of 1 번째 기사
4 of 2 번째 기사
4 of 3 번째 기사
4 of 4 번째 기사
4 of 

KeyboardInterrupt: 